In [7]:
import pandas as pd
import numpy as np
import folium

In [2]:
df=pd.read_csv(r'redistribution_center.txt')
df.head()

,center_name,latitude,longtitude
0,Dafz,25.256076,55.376633
1,Mirdif,25.217187,55.407012
2,Dubai Mall,25.198515,55.278802


In [5]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

def find_nearest_center(user_lat, user_lon, centers_df):
    distances = centers_df.apply(
        lambda row: haversine_distance(user_lat, user_lon, row['latitude'], row['longtitude']),
        axis=1
    )
    nearest_idx = distances.idxmin()
    return centers_df.iloc[nearest_idx]

# Example usage
user_location = (25.2048, 55.2708)  # Example coordinates
nearest = find_nearest_center(user_location[0], user_location[1], df)
print(f"Nearest center is {nearest['center_name']} at {nearest['latitude']}, {nearest['longtitude']}")

Nearest center is Dubai Mall at 25.198515, 55.278802


In [9]:
# Create a map centered on Dubai
dubai_map = folium.Map(location=[25.2048, 55.2708], zoom_start=11)

# Add marker for nearest redistribution center
folium.Marker(
    location=[nearest['latitude'], nearest['longtitude']],
    popup=nearest['center_name'],
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(dubai_map)

# Add marker for user location
folium.Marker(
    location=user_location,
    popup='Your Location',
    icon=folium.Icon(color='blue', icon='user')
).add_to(dubai_map)

# Draw line between user and nearest center
folium.PolyLine(
    locations=[
        user_location,
        [nearest['latitude'], nearest['longtitude']]
    ],
    weight=2,
    color='green',
    opacity=0.8
).add_to(dubai_map)

# Display the map
dubai_map